In [21]:
import numpy as np
import pandas as pd
from collections import Counter
import seaborn as sns

In [22]:
reviews = pd.read_csv('Datasets/reviews.csv')
reviews.drop('app_id', axis=1, inplace=True)
reviews.head()

,app_name,review_text,review_score,review_votes
0,Counter-Strike,Ruined my life.,1,0
1,Counter-Strike,This will be more of a ''my experience with th...,1,1
2,Counter-Strike,This game saved my virginity.,1,0
3,Counter-Strike,• Do you like original games? • Do you like ga...,1,0
4,Counter-Strike,"Easy to learn, hard to master.",1,1


In [23]:
reviews.sample(10)

,app_name,review_text,review_score,review_votes
1457234,Super Hexagon,"Simple, fresh, good music, enjoyable. Its amaz...",1,0
2226163,Game Dev Tycoon,"Haha spent so much time on this game, it may n...",1,0
4138176,We Happy Few,Early Access Review,1,1
4224585,SinaRun,Early Access Review,1,1
5908938,Five Nights at Freddy's: Sister Location,Five nights at Freddy's meets Portal 2. Nice.,1,0
558441,Crusader Kings II,Born too late to experience the middle ages as...,1,0
4392577,Deus Ex: Mankind Divided™,"If you liked the original Deus Ex, you gonna l...",1,0
1011664,Mark of the Ninja,Mark of the Ninja plays just right- everything...,1,0
5790394,Mount & Blade: Warband,played over the free weekend had lots of fun a...,1,0
924961,Starbound,Early Access Review,1,0


In [24]:
reviews.describe()

,review_score,review_votes
count,6.417106e+06,6.417106e+06
mean,6.394992e-01,1.472446e-01
std,7.687918e-01,3.543496e-01
min,-1.000000e+00,0.000000e+00
25%,1.000000e+00,0.000000e+00
50%,1.000000e+00,0.000000e+00
75%,1.000000e+00,0.000000e+00
max,1.000000e+00,1.000000e+00


In [25]:
print(reviews['review_score'].value_counts())
print()
print(reviews['review_votes'].value_counts())

 1    5260420
-1    1156686
Name: review_score, dtype: int64

0    5472222
1     944884
Name: review_votes, dtype: int64


### Weighted review score
In order to give a proper score for each game, it would not be appropriate to just calculate the average (since in the case where an average of 1.0 across 1 review is against an average of 0.95 across 100 reviews, the latter would be considered better). Therefore an appropriate formula need to be chosen to calculate this. This [blog post](https://steamdb.info/blog/steamdb-rating/) gives a solution to this.

In [26]:
positive_reviews = reviews[reviews['review_score']==1].groupby('app_name').count()

In [27]:
# count and mean of review_score_and_votes grouped by game
reviews_by_group = reviews.groupby(reviews["app_name"]).review_score.agg(["count","mean"])
positive_reviews = reviews[reviews['review_score']==1].groupby('app_name').count()['review_score']
reviews_by_group['positive_rate'] = positive_reviews/reviews_by_group['count']
reviews_by_group.sample(5)

,count,mean,positive_rate
app_name,,,
Sproggiwood,193,0.792746,0.896373
Saira,133,0.684211,0.842105
Bus-Simulator 2012,302,-0.304636,0.347682
Autocraft,172,0.302326,0.651163
Stacking,702,0.840456,0.920228


In [28]:
import math 

def calc_weighted_score(rate, total):
    return rate - (rate - 0.5) * math.pow(2, -math.log10(total+1))

weighted_score = []
for r, t in zip(reviews_by_group['positive_rate'], reviews_by_group['count']):
    weighted_score.append(calc_weighted_score(r, t))
reviews_by_group['weighted_score'] = weighted_score
reviews_by_group

,count,mean,positive_rate,weighted_score
app_name,,,,
神明的一天世界-God's One Day World,14,0.571429,0.785714,0.659272
! That Bastard Is Trying To Steal Our Gold !,26,-0.384615,0.307692,0.378996
"!""We Are The Dwarves""!",103,0.533981,0.766990,0.701026
"""Glow Ball"" - The billiard puzzle game",27,-0.629630,0.185185,0.300641
#SelfieTennis,63,0.492063,0.746032,0.675680
...,...,...,...,...
ルナティックドーン レジェンドパック,2,1.000000,1.000000,0.640795
ルナティックドーン 前途への道標,1,1.000000,1.000000,0.594164
丛林守望者（Ranger of the jungle）,5,0.600000,0.800000,0.625066


In [29]:
reviews_by_group.sort_values('weighted_score',ascending=False)[:10]

,count,mean,positive_rate,weighted_score
app_name,,,,
Portal 2,38924,0.975542,0.987771,0.967521
Sid Meier's Civilization V,8693,0.999310,0.999655,0.967083
Half-Life 2,6483,0.999075,0.999537,0.963967
Factorio,12553,0.977854,0.988927,0.960391
South Park™: The Stick of Truth™,3479,0.997700,0.998850,0.956010
Portal,16293,0.963174,0.981587,0.955602
The Binding of Isaac: Rebirth,23306,0.957264,0.978632,0.955444
Terraria,84828,0.941576,0.970788,0.955329
"Star Wars: Battlefront 2 (Classic, 2005)",2976,1.000000,1.000000,0.954995


In [32]:
reviews_by_group.to_csv('Datasets/reviews_processed.csv', index=True)